In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt


### Pre Processing

In [ ]:
dataset = pd.read_csv('tic-tac-toe.csv')
#dataset

In [ ]:
x=dataset.drop('class', axis=1)
y=dataset['class']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=1)


categorical_columns = x.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('scaler_num', StandardScaler(), x.columns.difference(categorical_columns)),
        ('onehot_catogrical', OneHotEncoder(), categorical_columns)
    ])


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    
])


x_scaled = pipeline.named_steps['preprocessor'].fit_transform(x)

# x_scaled
pd.DataFrame(x_scaled)


#### Train  Neural Network with one hidden layer (use number of hidden units ranging from 2-5) on the data sets using all the features

In [ ]:
hidden_units_range = range(2, 5)

models = {}

for hidden_units in hidden_units_range:

    model = MLPClassifier(hidden_layer_sizes=(hidden_units,), max_iter=1000, random_state=1, verbose=True)
   
  
    model_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    model_pipeline.fit(X_train, y_train)

    y_pred = model_pipeline.predict(X_test)

    
    accuracy = accuracy_score(y_test, y_pred)

    
    training_loss = model_pipeline.named_steps['classifier'].loss_curve_

    models[f'Model with {hidden_units} hidden units'] = {'model': model_pipeline, 'accuracy': accuracy, 'training_loss': training_loss}

for model_name, info in models.items():
    print(f'{model_name}: Accuracy = {info["accuracy"]:.2f}, Training Loss: {info["training_loss"]}')


####  Training using K-Nearest Neighbor 

In [ ]:

k_values_range = range(1, 6)

for k_value in k_values_range:
    knn_model = KNeighborsClassifier(n_neighbors=k_value, metric='euclidean')  # You can choose an appropriate distance metric
    
    knn_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', knn_model)
    ])

    
    knn_pipeline.fit(X_train, y_train)

    y_pred_knn = knn_pipeline.predict(X_test)

    accuracy_knn = accuracy_score(y_test, y_pred_knn)

    models[f'KNN Model with K={k_value}'] = {'model': knn_pipeline, 'accuracy': accuracy_knn}
    
for model_name, info in models.items():
    print(f'{model_name}: Accuracy = {info["accuracy"]:.2f}')


##### Train NN models by initializing weights in two different ways (i) initialize weight vector withzero values 


In [ ]:

for hidden_units in hidden_units_range:
    
    mlp_model_zero_init = MLPClassifier(
        hidden_layer_sizes=(hidden_units,),
        max_iter=1000,
        random_state=1,
        verbose=True,
        alpha=1e-10  # this will set a very small constant value for weight initialization almost zero 
    )
   
    mlp_pipeline_zero_init = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', mlp_model_zero_init)
    ])

    mlp_pipeline_zero_init.fit(X_train, y_train)

    y_pred_mlp_zero_init = mlp_pipeline_zero_init.predict(X_test)

    accuracy_mlp_zero_init = accuracy_score(y_test, y_pred_mlp_zero_init)

    models[f'MLP Model with {hidden_units} hidden units (Zero Initialization)'] = {
        'model': mlp_pipeline_zero_init,
        'accuracy': accuracy_mlp_zero_init
    }
for model_name, info in models.items():
    print(f'{model_name}: Accuracy = {info["accuracy"]:.2f}')


##### (ii) initialize weight vector with small random values (for model 1 and 2 only). See the effect of training by keeping the learning rate fix at 0.01. 

In [ ]:


plt.figure(figsize=(12, 8))
plt.title('Training Loss Over Iterations')


for i, hidden_units in enumerate(hidden_units_range):
    if i < 2:
       
        mlp_model = MLPClassifier(
            hidden_layer_sizes=(hidden_units,),
            max_iter=1000,
            random_state=1,
            verbose=True,
            alpha=1e-10  
        )
    else:
        
        mlp_model = MLPClassifier(
            hidden_layer_sizes=(hidden_units,),
            max_iter=1000,
            random_state=1,
            verbose=True
        )
        mlp_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', mlp_model)
    ])

    mlp_pipeline.fit(X_train, y_train)

    y_pred_mlp = mlp_pipeline.predict(X_test)

    accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

    models[f'MLP Model with {hidden_units} hidden units'] = {
        'model': mlp_pipeline,
        'accuracy': accuracy_mlp,
        'training_loss': mlp_pipeline.named_steps['classifier'].loss_curve_
    }

  
    plt.plot(models[f'MLP Model with {hidden_units} hidden units']['training_loss'], label=f'Model {i + 1}')

plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Training Loss')
plt.show()

for model_name, info in models.items():
    print(f'{model_name}: Accuracy = {info["accuracy"]:.2f}')


##### Runing models (1 and 2) using different learning rates 0.1,0.05,0.01,0.005,0.001 

In [ ]:

learning_rates = [0.1, 0.05, 0.01, 0.005, 0.001]


plt.figure(figsize=(12, 8))
plt.title('Training Loss Over Iterations (Different Learning Rates)')


for i, hidden_units in enumerate(hidden_units_range):
    for learning_rate in learning_rates:
        
        mlp_model = MLPClassifier(
            hidden_layer_sizes=(hidden_units,),
            max_iter=1000,
            random_state=1,
            verbose=True,
            learning_rate_init=learning_rate
        )

        mlp_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', mlp_model)
        ])

        
        mlp_pipeline.fit(X_train, y_train)
        y_pred_mlp = mlp_pipeline.predict(X_test)

        accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

        models[f'MLP Model with {hidden_units} hidden units and Learning Rate {learning_rate}'] = {
            'model': mlp_pipeline,
            'accuracy': accuracy_mlp,
            'training_loss': mlp_pipeline.named_steps['classifier'].loss_curve_
        }

        label = f'Model {i + 1}, LR {learning_rate}'
        plt.plot(models[f'MLP Model with {hidden_units} hidden units and Learning Rate {learning_rate}']['training_loss'], label=label)


plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Training Loss')
plt.show()
for model_name, info in models.items():
    print(f'{model_name}: Accuracy = {info["accuracy"]:.2f}')


##### Dividing data sets into three different sets namely training set, validation set and test set. 

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

hidden_units_range = range(2, 5)

learning_rates = [0.1, 0.05, 0.01, 0.005, 0.001]

plt.figure(figsize=(12, 8))
plt.title('Training Loss Over Iterations with Early Stopping')

models = {}

for i, hidden_units in enumerate(hidden_units_range):
    for learning_rate in learning_rates:
        
        mlp_model = MLPClassifier(
            hidden_layer_sizes=(hidden_units,),
            max_iter=1000,
            random_state=1,
            verbose=True,
            learning_rate_init=learning_rate,
            early_stopping=True,
            validation_fraction=0.1,  
            n_iter_no_change=10  
        )

        mlp_pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', mlp_model)
        ])

       
        mlp_pipeline.fit(X_train, y_train)

        y_val_pred = mlp_pipeline.predict(X_val)

        accuracy_val = accuracy_score(y_val, y_val_pred)

        models[f'MLP Model with {hidden_units} hidden units and Learning Rate {learning_rate}'] = {
            'model': mlp_pipeline,
            'accuracy_val': accuracy_val,
            'training_loss': mlp_pipeline.named_steps['classifier'].loss_curve_
        }

        label = f'Model {i + 1}, LR {learning_rate}, Val Acc {accuracy_val:.2f}'
        plt.plot(models[f'MLP Model with {hidden_units} hidden units and Learning Rate {learning_rate}']['training_loss'], label=label)

plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Training Loss')
plt.show()

for model_name, info in models.items():
    if "accuracy_val" in info:
        print(f'{model_name}: Validation Accuracy = {info["accuracy_val"]:.2f}')
    else:
        print(f'{model_name}: Validation Accuracy not available')

best_model = max(models, key=lambda x: models[x]['accuracy_val'])
final_model = models[best_model]['model']

y_test_pred = final_model.predict(X_test)

accuracy_test = accuracy_score(y_test, y_test_pred)
print(f'Best Model ({best_model}): Test Accuracy = {accuracy_test:.2f}')
